##🚀 Finetuning MLP on Fashion MNIST dataset (with W&B experiment tracking)

💡 In this example, we will finetune a simple MLP model on the fashion MNIST dataset and use the Weights & Biases experiments to track and monitior the training process. 

📌 The purpose of this example is to show all things possible with finetuner and give you a glimpse of its features.





### ⏰ Installing & Importing Dependencies

We will start this tutorial by installing the necessary ***pip*** dependencies.

In [ ]:
!pip install finetuner
!pip install torchvision

We will import the necessary dependencies.

In [ ]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR

from finetuner.toydata import generate_fashion
from finetuner.tuner.callback import WandBLogger
from finetuner.tuner.pytorch import PytorchTuner
from finetuner.tuner.pytorch.losses import TripletLoss
from finetuner.tuner.pytorch.miner import TripletEasyHardMiner

### 🔑 Authenticate W&B Dashboard

Before proceeding further, we need to install the W&B library and authenticate the account via login to track the expriements on the UI dashboard. 

You can find more details [here](https://docs.wandb.ai/quickstart).

In [ ]:
!pip install wandb

In [ ]:
!wandb login

### 🔨 Data Preprocessing (Using [DocArray](https://docarray.jina.ai/))

In this step we will load the training and evaluation(test) data usign the  [`generate_fashion()`](https://finetuner.jina.ai/api/finetuner.toydata/#finetuner.toydata.generate_fashion) helper function, which will produce a [Class Dataset](https://finetuner.jina.ai/basics/datasets/class-dataset/#class-dataset).

In [ ]:
train_data = generate_fashion()
eval_data = generate_fashion(is_testset=True)

Now, we will preprocess the data by adding some noise to it for the model to be trained properly.

In [ ]:
def preprocess_fn(doc: Document) -> np.ndarray:
    """Add some noise to the image"""
    new_image = doc.blob + np.random.normal(scale=0.01, size=doc.blob.shape)
    return new_image.astype(np.float32)

### ⚡ Create Model 

We will create a simple MLP [embedding model](https://finetuner.jina.ai/basics/glossary/#term-Embedding-model) using the Pytorch framework. 

In [ ]:
# create a MLP model
embed_model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=28 * 28, out_features=128),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=128, out_features=32),
)

We will create a fucntion to use `Adam` optimizer and dynamic scheduler for better learning rate and model training results.

In [ ]:
# Function to configure learning rate optimizer and scheduler
def configure_optimizer(model):
    optimizer = Adam(model.parameters(), lr=5e-4)
    scheduler = MultiStepLR(optimizer, milestones=[10, 20], gamma=0.5)

    return optimizer, scheduler

In [ ]:
# Creating the object for loss function and W&B callback
loss = TripletLoss(
    miner=TripletEasyHardMiner(pos_strategy='easy', neg_strategy='semihard')
)
logger_callback = WandBLogger()

### ⏳ Model Finetuning

We will create the `PytorchTuner` object and specify the training configuration. We will use the following configuration:


* `Triplet loss` function using hard miner with the easy positive and semi-hard negative strategy.
* `Adam` optimizer with initial learning rate of `0.0005`, which will be halved every `30` epochs
*  Tracking the experiement on Weights and Biases using `WandBLogger` callback.

In [ ]:
# Creating a tuner object for Pytorch model
tuner = PytorchTuner(
    embed_model,
    loss=loss,
    configure_optimizer=configure_optimizer,
    scheduler_step='epoch',
    callbacks=[logger_callback],
    device='cpu',
)

In [ ]:
# Fitting the tuner 
tuner.fit(
    train_data, eval_data, preprocess_fn=preprocess_fn, epochs=40, num_items_per_class=32
)

✨ You can monitor the training process by logging into your W&B account and watch the live updates there. 

Here’s an example of what you may see 👉

![](https://finetuner.jina.ai/_images/wandb.png)

